In [ ]:


# Load and clean customers_data.csv
customers_file = "customers_data.csv"  # Replace with your file path
customers = pd.read_csv(customers_file)
customers = customers.drop_duplicates(subset=['customer_id'])

In [11]:
import pandas as pd

# Load the products dataset
products_file = "products_data.csv"  # Replace with your file path
products = pd.read_csv(products_file)

# Check if the 'productPrice' column exists
if 'productPrice' in products.columns:
    # Convert all values in 'productPrice' to strings, then remove '$' and convert to float
    products['productPrice'] = products['productPrice'].astype(str).str.replace('$', '', regex=False)

    # Handle non-convertible values and convert to float
    products['productPrice'] = pd.to_numeric(products['productPrice'], errors='coerce')

    # Drop rows where conversion failed
    products = products.dropna(subset=['productPrice'])

    # Save cleaned products file
    products.to_csv("products_cleaned.csv", index=False)
    print("Cleaned 'products' dataset saved as 'products_cleaned.csv'")
else:
    print("'productPrice' column not found in the dataset.")


Cleaned 'products' dataset saved as 'products_cleaned.csv'


In [4]:
import pandas as pd
# Load and clean transactions_data.csv
transactions_file = "transactions.csv"  # Replace with your file path
transactions = pd.read_csv(transactions_file)

# Ensure the Order_Date column is properly formatted
if 'Order_Date' in transactions.columns:
    transactions['Order_Date'] = pd.to_datetime(transactions['Order_Date'], errors='coerce')
    transactions = transactions.dropna(subset=['Order_Date'])
else:
    transactions['Order_Date'] = pd.NaT  # Add placeholder if missing


In [ ]:
# Save cleaned datasets

transactions_cleaned_file = "transactions_cleaned.csv"


transactions.to_csv(transactions_cleaned_file, index=False)

print(f"Cleaned datasets saved as:\n{transactions_cleaned_file}")


Cleaned datasets saved as:
transactions_cleaned.csv


In [1]:
import mysql.connector
import pandas as pd
from datetime import datetime

# MySQL connection details
DB_HOST = 'localhost'
DB_USER = 'root'
DB_PASSWORD = 'raza'
DB_NAME = 'metro_dw'

# Function to insert data into the Time and Transactions tables
def insert_data_from_csv(csv_file_path):
    try:
        # Connect to MySQL database
        conn = mysql.connector.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME
        )
        cursor = conn.cursor()

        # Read the CSV file using pandas
        data = pd.read_csv(csv_file_path)

        # Prepare SQL queries
        time_insert_query = """
        INSERT INTO Time (Order_Date, Year, Month, Day, Weekday, Quarter)
        VALUES (%s, %s, %s, %s, %s, %s)
        """

        transaction_insert_query = """
        INSERT INTO Transactions (Order_ID, Order_Date, ProductID, customer_id, Quantity_Ordered, TOTAL_SALE, storeID, time_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """

        # Iterate through the rows of the CSV file and insert data
        for index, row in data.iterrows():
            # Extract data from CSV
            order_id = row['Order ID']
            order_date = datetime.strptime(row['Order Date'], '%Y-%m-%d %H:%M:%S')
            product_id = row['ProductID']
            quantity = row['Quantity Ordered']
            customer_id = row['customer_id']
            time_id = row['time_id']
            year = row['Year']
            month = row['Month']
            day = row['Day']
            weekday = row['Weekday']
            quarter = row['Quarter']

            # Insert data into Time table
            time_data = (order_date, year, month, day, weekday, quarter)
            cursor.execute(time_insert_query, time_data)

            # Get the auto-generated time_id from Time table
            cursor.execute("SELECT LAST_INSERT_ID()")
            time_id_inserted = cursor.fetchone()[0]

            # Calculate total sale (assuming you need to calculate it later, can be adjusted based on your data)
            total_sale = 0.0  # Replace with actual calculation logic if needed

            # Insert data into Transactions table
            transaction_data = (order_id, order_date, product_id, customer_id, quantity, total_sale, 1, time_id_inserted)  # storeID=1 (or set it accordingly)
            cursor.execute(transaction_insert_query, transaction_data)

        # Commit the transactions
        conn.commit()
        print(f"{len(data)} rows inserted successfully into the database!")

    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        cursor.close()
        conn.close()

# Example usage
insert_data_from_csv('transactions.csv')


30247 rows inserted successfully into the database!


In [ ]:
import mysql.connector
import pandas as pd
from datetime import datetime

# MySQL connection details
DB_HOST = 'localhost'
DB_USER = 'root'
DB_PASSWORD = 'raza'
DB_NAME = 'metro_dw'

# Function to insert data into Suppliers, Stores, Products, Customers, Time, and Transactions tables
def insert_data_from_csv(csv_file_path):
    try:
        # Connect to MySQL database
        conn = mysql.connector.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME
        )
        cursor = conn.cursor()

        # Read the CSV file using pandas
        data = pd.read_csv(csv_file_path)

        # Determine which table to insert based on the columns in the CSV
        if 'ProductID' in data.columns:
            insert_query = """
            INSERT INTO Products (ProductID, productName, productPrice, supplierID, storeID)
            VALUES (%s, %s, %s, %s, %s)
            """
            for index, row in data.iterrows():
                product_data = (row['ProductID'], row['productName'], row['productPrice'], row['supplierID'], row['storeID'])
                cursor.execute(insert_query, product_data)

        elif 'customer_id' in data.columns:
            insert_query = """
            INSERT INTO Customers (customer_id, customer_name, gender)
            VALUES (%s, %s, %s)
            """
            for index, row in data.iterrows():
                customer_data = (row['customer_id'], row['customer_name'], row['gender'])
                cursor.execute(insert_query, customer_data)

        elif 'Order ID' in data.columns:
            # Time and Transactions
            time_insert_query = """
            INSERT INTO Time (Order_Date, Year, Month, Day, Weekday, Quarter)
            VALUES (%s, %s, %s, %s, %s, %s)
            """
            transaction_insert_query = """
            INSERT INTO Transactions (Order_ID, Order_Date, ProductID, customer_id, Quantity_Ordered, TOTAL_SALE, storeID, time_id)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """

            for index, row in data.iterrows():
                # Insert data into Time table
                order_date = datetime.strptime(row['Order Date'], '%Y-%m-%d %H:%M:%S')
                year = row['Year']
                month = row['Month']
                day = row['Day']
                weekday = row['Weekday']
                quarter = row['Quarter']
                time_data = (order_date, year, month, day, weekday, quarter)
                cursor.execute(time_insert_query, time_data)

                # Get the auto-generated time_id from Time table
                cursor.execute("SELECT LAST_INSERT_ID()")
                time_id_inserted = cursor.fetchone()[0]

                # Insert data into Transactions table
                order_id = row['Order ID']
                product_id = row['ProductID']
                quantity = row['Quantity Ordered']
                customer_id = row['customer_id']
                total_sale = 0.0  # You can modify this based on your business logic to calculate total sale
                transaction_data = (order_id, order_date, product_id, customer_id, quantity, total_sale, 1, time_id_inserted)
                cursor.execute(transaction_insert_query, transaction_data)

        # Commit the transactions
        conn.commit()
        print(f"{len(data)} rows inserted successfully into the database!")

    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        cursor.close()
        conn.close()

# Example usage: Insert data into all tables
#insert_data_from_csv('products.csv')  # Insert Products data
#insert_data_from_csv('customers.csv')  # Insert Customers data
#insert_data_from_csv('transactions.csv')  # Insert Transactions and Time data


Error: 1062 (23000): Duplicate entry '17' for key 'customers.PRIMARY'
